In [1]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import least_squares

In [2]:
R = 8.3144626 #J/mol-K
F = 96485.33212 #C/mol

### Butler Volmer equations to describe kinetics
    i_0 = exchange current density - the value of the current density at equilibrium
    alpha_a = anodic charge transfer coefficient
    alpha_c = cathodic charge transfer coefficient
    alpha_a + alpha_c = number of electrons transferred in reaction
    eta = surface overpotential = potential - equilibrium potential

In [3]:
#General Butler-Volmer equation
def bv_eq(i_0, alpha_a, alpha_c, eta):
    i = i_0*(np.exp((alpha_a*F*eta)/(R*T)) - np.exp((-alpha_c*F*eta)/(R*T)))
    return i